In [25]:
from generate_signal import generate_X_matrix
from beamforming import beamforming_method
from generate_signal import generate_A_matrix
from generate_signal import generate_noise
from generate_signal import generate_R_hat
from music import music_method
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np


nbSources = 2 # Nombre de sources
nbSensors = 10 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels
signal_noise_ratio = 3 # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
var1 = 1 # Variance du signal 1
var2 = 1 # Variance du signal 2
L = 10
T = 100
correlation_List = [0.4] # Liste des corrélations. Il y a une corrélation nécéssaire pour chaque paire distincte de sources différentes: 0 pour 1 source, 1 pour 2 sources, 3 pour 3 sources, 6 pour 4 sources etc...
# Ordre de remplisage de la correlation_List: de gauche à droite et ligne par ligne, moitié haut-droite de la matrice uniquement, puis symétrie de ces valeurs pour la moitié bas-gauche.


In [27]:
num_samples = 100  # Nombre d'échantillons à générer

# Initialiser les tableaux pour stocker les matrices de covariance et les angles estimés
all_R_hat = np.zeros((num_samples, L, L), dtype=complex)
all_estimated_angles = []
for i in range(num_samples):
    # Générer des angles aléatoires pour chaque échantillon
    theta1 = np.random.uniform(-90, 90)
    theta2 = np.random.uniform(-90, 90)
    var1 = np.random.uniform(0,1000000000000)
    var2 = np.random.uniform(0,1000000000000)
    thetaList = [theta1, theta2]
    varList = [var1, var2]

    # Générer la matrice X
    X = generate_X_matrix(nbSources, L, T, thetaList, varList, correlation_List, signal_noise_ratio)

    # Calcul de la matrice de covariance pour l'échantillon actuel
    R_hat = generate_R_hat(X)
    all_R_hat[i] = R_hat

    # Calcul du spectre MUSIC pour l'échantillon actuel
    estimated_angles = music_method(X, L, T, print_angles = False, draw_plot = False)
    all_estimated_angles.append(estimated_angles)

# Division des données en ensembles d'entraînement et de test
all_estimated_angles = np.array(all_estimated_angles)  # Conversion en numpy array si nécessaire
R_hat_train, R_hat_test, angles_train, angles_test = train_test_split(
    all_R_hat, all_estimated_angles, test_size=0.2, random_state=42
)

[[6.49554894e+11 5.66342769e+10]
 [5.66342769e+10 3.08619155e+10]]
[[7.11533265e+11 2.05348173e+11]
 [2.05348173e+11 3.70396177e+11]]
[[2.60725627e+11 8.25506900e+10]
 [8.25506900e+10 1.63356986e+11]]
[[7.91799000e+10 9.50986814e+10]
 [9.50986814e+10 7.13861662e+11]]
[[4.71736773e+11 2.00692382e+11]
 [2.00692382e+11 5.33632243e+11]]
[[2.65410624e+11 1.37310487e+11]
 [1.37310487e+11 4.43985853e+11]]
[[7.88005750e+11 5.25569265e+10]
 [5.25569265e+10 2.19083944e+10]]
[[5.18636018e+11 2.13085453e+11]
 [2.13085453e+11 5.47173365e+11]]
[[5.41694019e+11 2.50066220e+11]
 [2.50066220e+11 7.21499503e+11]]
[[3.46623504e+11 4.60514725e+10]
 [4.60514725e+10 3.82392223e+10]]
[[7.75185367e+11 3.43479939e+11]
 [3.43479939e+11 9.51211747e+11]]
[[8.49802492e+10 6.24057937e+10]
 [6.24057937e+10 2.86425605e+11]]
[[4.60976274e+11 2.43874017e+11]
 [2.43874017e+11 8.06366555e+11]]
[[3.18903862e+11 1.11028628e+11]
 [1.11028628e+11 2.41596245e+11]]
[[6.64455391e+10 3.28291280e+10]
 [3.28291280e+10 1.01375471e+